In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.datasets import make_classification
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, auc, accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
#사고율이 10을 넘는 관측치는 이상치로 판단해 모델링 과정에서 제거
mdl_df=df[df['사고율']<=10]
mdl_df['사고율'].describe()

In [ ]:
#사고율이 높은 고객 탐지 성능 향상을 위해 고위험군으로 의심되는 고객을 선별적으로 분류하는 이진 분류 모델링 진행
mdl_df['고위험']=0
mdl_df.loc[mdl_df['사고율']>=1,'고위험']=1

In [ ]:
#CV를 위해 데이터셋 분리
mdl_df=mdl_df.drop(['C','O'],axis=1)
X=mdl_df.drop(['고위험'],axis=1)
y=mdl_df['고위험']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=831)

In [ ]:
tree_y_train=X_train['사고율']
X=X_train.drop('사고율',axis=1)
tree_y_test=X_test['사고율']
X_test=X_test.drop('사고율',axis=1)
y=y_train

In [ ]:
#One-hot Encoding
#파이썬 logistic regression은 범주형 자료에 대해 따로 One-hot Encoding을 진행해주지 않기 때문
X = pd.get_dummies(X, columns = ['J'], drop_first = False)
X_test = pd.get_dummies(X_test, columns = ['J'], drop_first = False)

In [ ]:
#train-valid set 분리를 통해 train data로 성능 평가 진행 및 최적의 임계점 선정
log_X_train, log_X_valid, log_y_train, log_y_valid = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=831)

In [ ]:
#logistic regression 유의한 변수를 파악하기 위해 다중공선성이 높은 변수 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor
def vif_calculate(data):
    vif=pd.DataFrame()
    vif["feature"] = data.columns
    vif["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    return vif

In [ ]:
vif_calculate(log_X_train)

In [ ]:
#다중공선성 높은 변수 제거 
log_X_train.drop(['J_4'],axis=1,inplace=True)
log_X_valid.drop(['J_4'],axis=1,inplace=True)

In [ ]:
vif_calculate(log_X_train)

In [ ]:
#다중공선성 높은 변수 제거 
log_X_train.drop(['K'],axis=1,inplace=True)
log_X_valid.drop(['K'],axis=1,inplace=True)

In [ ]:
vif_calculate(log_X_train)

In [ ]:
X_with_const = sm.add_constant(log_X_train)

model = sm.Logit(log_y_train, X_with_const)
result = model.fit()

print(result.summary())

In [ ]:
#로지스틱 회귀에서 유의하지 않은 변수 제거
log_X_train.drop(['J_6','J_10','J_11'],axis=1,inplace=True)
log_X_valid.drop(['J_6','J_10','J_11'],axis=1,inplace=True)

In [ ]:
X_with_const = sm.add_constant(log_X_train)

model = sm.Logit(log_y_train, X_with_const)
result = model.fit()

print(result.summary())

In [ ]:
#로지스틱 회귀 모델 학습
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(log_X_train, log_y_train)

In [ ]:
#valid set으로 예측
y_pred = model.predict(log_X_valid)
y_prob = model.predict_proba(log_X_valid)[:, 1] 
fpr, tpr, thresholds = roc_curve(log_y_valid, y_prob)

In [ ]:
#ROC 확인
roc_auc = auc(fpr, tpr)
roc_auc

In [ ]:
#ROC curve 시각화
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

In [ ]:
#분류 성능을 극대화시키는 분류 임계점 찾기
optimal_idx=np.argmax(tpr-fpr)
optimal=thresholds[optimal_idx]

optimal

In [ ]:
#test set에 동일한 전처리 진행
X.drop(['K','J_4','J_6','J_10','J_11',],axis=1,inplace=True)
X_test.drop(['K','J_4','J_6','J_10','J_11',],axis=1,inplace=True)

In [ ]:
#전체 train data로 재학습
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X, y)